# Import Libraries

In [7]:
''' import relevant libaries for manipulating data'''
import requests
import pandas as pd
import pycountry
import copy
import numpy as np

pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', None)

import coinoxr
from coinoxr import Historical
from coinoxr import Currency
from coinoxr.requestor import Requestor

coinoxr.app_id = "114d34f7c3164eacba5c42442ae8c4c5"

# Get All Currencies Names and Codes

In [58]:
cur = Currency().get()
currencies = dict(cur.body)
cur

Response(code=200, body={'AED': 'United Arab Emirates Dirham', 'AFN': 'Afghan Afghani', 'ALL': 'Albanian Lek', 'AMD': 'Armenian Dram', 'ANG': 'Netherlands Antillean Guilder', 'AOA': 'Angolan Kwanza', 'ARS': 'Argentine Peso', 'AUD': 'Australian Dollar', 'AWG': 'Aruban Florin', 'AZN': 'Azerbaijani Manat', 'BAM': 'Bosnia-Herzegovina Convertible Mark', 'BBD': 'Barbadian Dollar', 'BDT': 'Bangladeshi Taka', 'BGN': 'Bulgarian Lev', 'BHD': 'Bahraini Dinar', 'BIF': 'Burundian Franc', 'BMD': 'Bermudan Dollar', 'BND': 'Brunei Dollar', 'BOB': 'Bolivian Boliviano', 'BRL': 'Brazilian Real', 'BSD': 'Bahamian Dollar', 'BTC': 'Bitcoin', 'BTN': 'Bhutanese Ngultrum', 'BWP': 'Botswanan Pula', 'BYN': 'Belarusian Ruble', 'BZD': 'Belize Dollar', 'CAD': 'Canadian Dollar', 'CDF': 'Congolese Franc', 'CHF': 'Swiss Franc', 'CLF': 'Chilean Unit of Account (UF)', 'CLP': 'Chilean Peso', 'CNH': 'Chinese Yuan (Offshore)', 'CNY': 'Chinese Yuan', 'COP': 'Colombian Peso', 'CRC': 'Costa Rican Colón', 'CUC': 'Cuban Convert

# Import Nationality and Currency-Nationality Mapping Data
## Remove Currencies without Representation at Minerva

In [140]:
#https://gist.github.com/emmastiefel/c891ff0910a253b1343259c5f27510bd
nationality =  pd.read_csv("data/nationality.csv")
nationality_df = nationality.loc[:, nationality.columns.intersection(['Nationality', 'Total Students'])]


#https://gist.github.com/HarishChaudhari/4680482
mapping =  pd.read_csv("data/map.csv")

c_list = []
for count, co in enumerate(mapping["Country"]):
    if str(co) not in list(nationality_df["Nationality"]):
        c_list.append(count)

mapping = mapping.drop(c_list)

nationality_df = nationality_df.merge(mapping, left_on='Nationality', right_on='Country').drop(columns = ['Country'])

nationality_df = nationality_df.rename(columns={"Code": "Currency Code"})

nationality_df

,Nationality,Total Students,Currency Code
0,Albania,1,ALL
1,Algeria,1,DZD
2,Argentina,3,ARS
3,Armenia,3,AMD
4,Australia,3,AUD
...,...,...,...
72,Uruguay,2,UYU
73,Uzbekistan,1,UZS
74,Venezuela,2,VEF
75,Vietnam,35,VND


## Map Number of Students to Currencies

In [376]:
#multiple countries to same currency, use groupby sum
student_df = nationality_df.groupby(['Currency Code'], as_index=False)['Total Students'].sum()
student_df = student_df.drop(index = 62) #drop venezula, 2 students unaccounted for.
student_df

,Currency Code,Total Students
0,AED,1
1,ALL,1
2,AMD,3
3,ARS,3
4,AUD,3
...,...,...
60,UYU,2
61,UZS,1
63,VND,35
64,ZAR,4


In [142]:
currencies = dict(cur.body)
no_need = list(set(currencies.keys()) - set(student_df['Currency Code']))
for currency in no_need: 
    currencies.pop(currency)
len(currencies)

65

In [277]:
df = pd.DataFrame.from_dict(currencies, orient='index', columns = ['CurrencyName'])
df["Students"] = list(student_df['Total Students'])
df

,CurrencyName,Students
AED,United Arab Emirates Dirham,1
ALL,Albanian Lek,1
AMD,Armenian Dram,3
ARS,Argentine Peso,3
AUD,Australian Dollar,3
...,...,...
UYU,Uruguayan Peso,2
UZS,Uzbekistan Som,1
VND,Vietnamese Dong,35
ZAR,South African Rand,4


# Get Historical Forex from API

## By Semester

In [144]:
# 17 Fall July 01, 2017 - $13225.00 (includes security deposit)
# 18 Spring December 01, 2017 - $12225.00

# 18 Fall July 06, 2018 - $12761.60 (includes insurance)
# 19 Spring November 30, 2018 - $12761.60 (includes insurance)

# 19 Fall June 30, 2019 - $12975
# 20 Spring November 30, 2019 - $12975

# 20 Fall July 15, 2020 - $12975
# 21 Spring November 30, 2020 - $16925


'''
Fall_17 = Historical().get("2017-07-01")
Spring_18 = Historical().get("2017-12-01")
Fall_18 = Historical().get("2018-07-06")
Spring_19 = Historical().get("2018-11-30")
Fall_19 = Historical().get("2019-06-30")
Spring_20 = Historical().get("2019-11-30")
Fall_20 = Historical().get("2020-07-15")
Spring_21 = Historical().get("2020-11-30")

responses = [Fall_17.body['rates']
             , Spring_18.body['rates']
             , Fall_18.body['rates']
             , Spring_19.body['rates']
             ,Fall_19.body['rates']
             ,Spring_20.body['rates']
             ,Fall_20.body['rates']
             ,Spring_21.body['rates'] ]

RES = copy.deepcopy(responses)

sems = ['Fall2017', 'Spring2018', 'Fall2018','Spring2019',
        'Fall2019','Spring2020', 'Fall2020','Spring2021']

#includes housing, student services, tuition and manifest (last sem)
#excludes security deposits, insurance payments, etc.
sem_rates = [12225, 12225, 12475, 12475,
            12975, 12975,12975, 12975]
'''

'\nFall_17 = Historical().get("2017-07-01")\nSpring_18 = Historical().get("2017-12-01")\nFall_18 = Historical().get("2018-07-06")\nSpring_19 = Historical().get("2018-11-30")\nFall_19 = Historical().get("2019-06-30")\nSpring_20 = Historical().get("2019-11-30")\nFall_20 = Historical().get("2020-07-15")\nSpring_21 = Historical().get("2020-11-30")\n\nresponses = [Fall_17.body[\'rates\']\n             , Spring_18.body[\'rates\']\n             , Fall_18.body[\'rates\']\n             , Spring_19.body[\'rates\']\n             ,Fall_19.body[\'rates\']\n             ,Spring_20.body[\'rates\']\n             ,Fall_20.body[\'rates\']\n             ,Spring_21.body[\'rates\'] ]\n\nRES = copy.deepcopy(responses)\n\nsems = [\'Fall2017\', \'Spring2018\', \'Fall2018\',\'Spring2019\',\n        \'Fall2019\',\'Spring2020\', \'Fall2020\',\'Spring2021\']\n\n#includes housing, student services, tuition and manifest (last sem)\n#excludes security deposits, insurance payments, etc.\nsem_rates = [12225, 12225, 12

## By Calender Year

In [145]:
_17 = Historical().get("2017-12-31")
_18 = Historical().get("2018-12-31")
_19 = Historical().get("2019-12-31")
_20 = Historical().get("2020-12-31")

responses = [_17.body['rates']
             ,_18.body['rates']
             ,_19.body['rates']
             ,_20.body['rates']]

RES = copy.deepcopy(responses)

sems = ['2017','2018','2019', '2020']

#includes housing, student services, tuition and manifest (last sem)
#excludes security deposits, insurance payments, etc.
sem_rates = [24450, 24950, 25950, 25950]

# Inflation Data!

In [307]:
#enter data
inflation_df =  pd.read_csv("data/imf_inflation.csv")

#store last row of euro data
euro_inflation_data = inflation_df.iloc[-1]
euro_inflation_data['Currency Code'] = "EUR"

inflation_df 



,Country,2017,2018,2019,2020
0,Afghanistan,3,0.8,2.8,5
1,Albania,1.8,1.8,1.1,1.2
2,Algeria,4.9,2.7,2.4,5.2
3,Angola,23.7,18.6,16.9,22.2
4,Antigua and Barbuda,2.4,1.7,1.5,0.7
...,...,...,...,...,...
190,West Bank and Gaza,0,0.3,1.3,-1.7
191,Yemen,47,14.3,6.2,45.4
192,Zambia,6.1,7.9,11.7,13
193,Zimbabwe,3.4,42.1,521.1,495


In [308]:
#find rows to drop in inflation dataset
inflation_data_to_drop = []
for index,country in enumerate(inflation_df.Country):
    if country not in list(nationality_df.Nationality):
        inflation_data_to_drop.append(index)
        
#drop rows with no Minervan nationality
inflation_df = inflation_df.drop(inflation_data_to_drop)
inflation_df = inflation_df.reset_index(drop=True)

#merge on nationality_df to find currencies of each country
inflation_df = inflation_df.merge(nationality_df, left_on='Country', right_on='Nationality').drop(columns = ['Nationality','Total Students'])
inflation_df

,Country,2017,2018,2019,2020,Currency Code
0,Albania,1.8,1.8,1.1,1.2,ALL
1,Algeria,4.9,2.7,2.4,5.2,DZD
2,Argentina,24.8,47.6,53.8,41.7,ARS
3,Armenia,2.6,1.8,0.7,1.5,AMD
4,Australia,2.1,1.7,1.8,0.6,AUD
...,...,...,...,...,...,...
72,Uruguay,6.6,8,8.8,9.5,UYU
73,Uzbekistan,18.8,14.3,15.2,12.1,UZS
74,Venezuela,862.6,130060.2,9585.5,6500,VEF
75,Vietnam,2.6,3,5.2,3.1,VND


In [309]:
more_inflation_data_to_drop = []
for index,currency in enumerate(inflation_df["Currency Code"]):
    if currency == "EUR":
        more_inflation_data_to_drop.append(index)
        
#drop Euro zone country respective inflation data
inflation_df = inflation_df.drop(more_inflation_data_to_drop)
inflation_df = inflation_df.reset_index(drop=True)

#drop venezuela because that inflation rate wow.
inflation_df = inflation_df.drop([62])

#include EURO
inflation_df = inflation_df.append(euro_inflation_data).drop(columns = ['Country'])

#convert to floats
inflation_df[['2017','2018','2019','2020']] = inflation_df[['2017','2018','2019','2020']].astype('float')

#convert to percentages
#inflation_df[['2017','2018','2019','2020']] = inflation_df[['2017','2018','2019','2020']].apply(lambda x: x/100)

#sort alphabetic currency code
inflation_df = inflation_df.sort_values('Currency Code', ascending=True)

#reset index
inflation_df = inflation_df.reset_index(drop=True)


inflation_df.index = inflation_df['Currency Code']


inflation_df

,2017,2018,2019,2020,Currency Code
Currency Code,,,,,
AED,2.00000,3.10000,-1.90000,-1.50000,AED
ALL,1.80000,1.80000,1.10000,1.20000,ALL
AMD,2.60000,1.80000,0.70000,1.50000,AMD
ARS,24.80000,47.60000,53.80000,41.70000,ARS
AUD,2.10000,1.70000,1.80000,0.60000,AUD
...,...,...,...,...,...
UYU,6.60000,8.00000,8.80000,9.50000,UYU
UZS,18.80000,14.30000,15.20000,12.10000,UZS
VND,2.60000,3.00000,5.20000,3.10000,VND


# Convert to 2017 USD

In [294]:
explore = df.loc[:, df.columns.intersection([ sem for sem in sems ])]
#explore = explore.drop(index = "VEF")

#bar chart data
explore.columns = sems

explore['Currency'] = explore.index
first_col = explore.pop('Currency')
exlore =explore.insert(0, 'Currency', first_col)
explore["FullCurrency"] = df["CurrencyName"]
explore['total'] = explore.sum(axis=1)

#explore = explore.sort_values('total', ascending=False)

explore = explore.drop(columns = 'total')
#explore.to_csv("./ForexQuestIdyll/data/explore2.csv",index = False)

explore


,Currency,2017,2018,2019,2020,FullCurrency
AED,AED,24450,24949,25952,25951,United Arab Emirates Dirham
ALL,ALL,24450,24304,25423,23591,Albanian Lek
AMD,AMD,24450,25064,25716,28079,Armenian Dram
ARS,ARS,24450,48997,81073,113905,Argentine Peso
AUD,AUD,24450,27675,28887,26320,Australian Dollar
...,...,...,...,...,...,...
UYU,UYU,24450,28081,33537,38063,Uruguayan Peso
UZS,UZS,24450,25681,30422,33524,Uzbekistan Som
VND,VND,24450,25406,26497,26512,Vietnamese Dong
ZAR,ZAR,24450,28946,29372,30738,South African Rand


In [291]:
for count,sem in enumerate(RES):

    for c in list(sem.keys()):
        if c not in list(currencies.keys()):
            sem.pop(c)

    #get rates for each currency        
    df[sems[count] +"_Rates"] = sem.values()
    
    #usd value in 2017
    #multiply exchange rate of that year with tuition, convert to 
    df[sems[count]] = [ round(((rate * sem_rates[count])/ list(RES[0].values())[index])) for index,rate in enumerate(sem.values()) ]
    
    #df[sems[count] +"_Difference"] = [ round(((rate * sem_rates[count])/ list(RES[0].values())[index]  - sem_rates[0])) for index,rate in enumerate(sem.values()) ]

df

,CurrencyName,Students,2017_Rates,2017,2018_Rates,2018,2019_Rates,2019,2020_Rates,2020
AED,United Arab Emirates Dirham,1,3.67287,24450,3.67278,24949,3.67320,25952,3.67301,25951
ALL,Albanian Lek,1,110.81531,24450,107.94662,24304,108.56339,25423,100.74137,23591
AMD,Armenian Dram,3,483.25497,24450,485.46483,25064,478.89154,25716,522.91000,28079
ARS,Argentine Peso,3,19.15400,24450,37.61500,48997,59.84070,81073,84.07466,113905
AUD,Australian Dollar,3,1.28060,24450,1.42048,27675,1.42552,28887,1.29885,26320
...,...,...,...,...,...,...,...,...,...,...
UYU,Uruguayan Peso,2,28.79468,24450,32.40837,28081,37.21296,33537,42.23501,38063
UZS,Uzbekistan Som,1,8108.25000,24450,8345.89174,25681,9505.63945,30422,10474.79528,33524
VND,Vietnamese Dong,35,22701.41115,24450,23116.20792,25406,23179.64399,26497,23192.97046,26512
ZAR,South African Rand,4,12.37860,24450,14.36125,28946,14.01090,29372,14.66265,30738


# Adjust For Inflation

In [371]:
'''how to adjust for inflation with inflation rates
inflation rate = cpi x year - cpi base year

'''
cpi17 = 100
cpi18 = ((inflation_df['2018']*cpi17/100)+cpi17)
cpi19 = ((inflation_df['2019']*cpi18/100)+cpi18)
cpi20 = ((inflation_df['2020']*cpi19/100)+cpi19)

explore['2017'] = explore['2017'] / cpi17 * cpi17
explore['2018'] = explore['2018'] / cpi18 * cpi17
explore['2019'] =  explore['2019'] / cpi19 * cpi18
explore['2020'] = explore['2020'] / cpi20 * cpi19

In [409]:
#line chart data

explore['Currency'] =  df["CurrencyName"]
explore = explore.drop(columns = 'FullCurrency')
explore = explore.sort_values('Currency', ascending=True)
explore.to_csv("./ForexQuestIdyll/data/explore.csv",index = False)

import csv

with open("./ForexQuestIdyll/data/currencies.csv", 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(list(explore.Currency))

explore

,Currency,Fall2017,Spring2018,Fall2018,Spring2019,Fall2019,Spring2020,Fall2020,Spring2021
ALL,Albanian Lek,12225,11843,11549,11803,12060,12456,12163,11599
DZD,Algerian Dinar,12225,13013,13586,13757,14287,14481,15454,15558
ARS,Argentine Peso,12225,12659,20936,28316,33178,46775,55644,63463
AMD,Armenian Dram,12225,12376,12560,12625,12935,12916,13040,13040
AUD,Australian Dollar,12225,12350,12910,13118,14205,14758,14236,13557
...,...,...,...,...,...,...,...,...,...
USD,United States Dollar,12225,12225,12475,12475,12975,12975,12975,12975
UYU,Uruguayan Peso,12225,12522,13863,14138,16158,17423,20086,19543
UZS,Uzbekistan Som,12225,25067,24780,26198,28130,31181,33536,34133
VND,Vietnamese Dong,12225,12230,12624,12768,13342,13218,13273,13278


In [410]:
explore = explore.sort_index(ascending=True)

#to observable to convert
sems_dates = ["July 1, 2017",
              "December 01, 2017",
              "July 6, 2018",
              "November 30, 2018",
              "June 30, 2019",
              "November 30, 2019",
              "July 15, 2020",
              "November 30, 2020",]

a = ['Currency'] + sems_dates  

explore.columns = a
explore['Students'] = list(df['Students'])
explore.to_csv("data/observable.csv", index=False)

explore

,Currency,"July 1, 2017","December 01, 2017","July 6, 2018","November 30, 2018","June 30, 2019","November 30, 2019","July 15, 2020","November 30, 2020",Students
AED,United Arab Emirates Dirham,12225,12225,12475,12476,12975,12975,12976,12976,1
ALL,Albanian Lek,12225,11843,11549,11803,12060,12456,12163,11599,1
AMD,Armenian Dram,12225,12376,12560,12625,12935,12916,13040,13040,3
ARS,Argentine Peso,12225,12659,20936,28316,33178,46775,55644,63463,3
AUD,Australian Dollar,12225,12350,12910,13118,14205,14758,14236,13557,3
...,...,...,...,...,...,...,...,...,...,...
UYU,Uruguayan Peso,12225,12522,13863,14138,16158,17423,20086,19543,2
UZS,Uzbekistan Som,12225,25067,24780,26198,28130,31181,33536,34133,1
VND,Vietnamese Dong,12225,12230,12624,12768,13342,13218,13273,13278,35
ZAR,South African Rand,12225,12828,12832,13210,13949,14516,16428,15297,4
